# Post-processing modification example - TrioAPI Python package

This is a simple notebook illustrating how to manage problem postprocessing using TrioAPI.



## Dataset collection

As shown in the first notebook, we have to import trioapi, collect the dataset and the problem.

In [4]:
import trioapi as ta
dataset = ta.get_jdd("Cas_turbulence",".")
pb = dataset.get(ta.PROBLEM)

We also import trustify_gen_pyd to manage generated classes.

In [5]:
import trioapi.trustify_gen_pyd as tgp

pb is a python object so we can access to its attributes and here we want to modify postoprocessing.

In [6]:
pb_postprocessing=pb.postraitement

## Modification

For postprocessing there are a lot of attributes, some are classic values which can be modified directly like in the basic scheme modification notebook. Other are complex python class which can be managed using functions in the API.

### Simple modification

We can for example modify the format of post processing file.

In [7]:
pb_postprocessing.format = 'lata'

### Probe modification

We can use the API to create points and segments python objects for probes. \
For points we have to give a list of coordinates for each points (list of 2 arguments). \
For segments we have to give the number of points and the starting and ending coordinates.

In [8]:
points = ta.create_probe_points([[0.5,1],[2,1],[0.8,0.5]])
segment = ta.create_probe_segment(50,[0,0.8],[1,1.2])

Then we can create 2 probes : 

In [9]:
probe1 = ta.create_probe('probe1',None,'vitesse','0.05',points)
probe2 = ta.create_probe('probe2',None,'pression','0.005',segment)

Now we can add them to the problem:

In [10]:
ta.add_probe(pb,probe1)
ta.add_probe(pb,probe2)

We can check it in the problem.

In [11]:
print(pb_postprocessing.sondes)

[Sonde(nom_sonde='nut_max', special=None, nom_inco='nut_max', mperiode='periode', prd=1e-06, type=Numero_elem_sur_maitre(numero=0)), Sonde(nom_sonde='vitesse_axe', special=None, nom_inco='vitesse', mperiode='periode', prd=1e-06, type=Segment(nbr=100, point_deb=Un_point(pos=[-10.87, 4.2]), point_fin=Un_point(pos=[0.0, 4.2]))), Sonde(nom_sonde='keps_axe', special=None, nom_inco='k_eps', mperiode='periode', prd=1e-06, type=Segment(nbr=100, point_deb=Un_point(pos=[-10.87, 4.2]), point_fin=Un_point(pos=[0.0, 4.2]))), Sonde(nom_sonde='u_x-6', special=None, nom_inco='vitesse', mperiode='periode', prd=1e-06, type=Segment(nbr=10, point_deb=Un_point(pos=[-6.0, 0.0]), point_fin=Un_point(pos=[-6.0, 4.7]))), Sonde(nom_sonde='u_x03', special=None, nom_inco='vitesse', mperiode='periode', prd=1e-06, type=Segment(nbr=10, point_deb=Un_point(pos=[3.0, 0.0]), point_fin=Un_point(pos=[3.0, 4.7]))), Sonde(nom_sonde='u_x06', special=None, nom_inco='vitesse', mperiode='periode', prd=1e-06, type=Segment(nbr=10,

There is the list of probes and we can see that the two last are the on that we just added. \
If we want to change a point we can do it like that :

In [12]:
index=ta.get_probe_index_by_name(pb,"probe1")
new_points  = ta.create_probe_points([[0.8,2],[1,1.3],[1,0]])
pb_postprocessing.sondes[index].type= new_points
print(pb_postprocessing.sondes[index])

nom_sonde='probe1' special=None nom_inco='vitesse' mperiode='periode' prd=0.05 type=Points(points=[Un_point(pos=[0.8, 2.0]), Un_point(pos=[1.0, 1.3]), Un_point(pos=[1.0, 0.0])])


### Fields to be postprocessed modification

The API also provides function to add fields to be postprocessed in the list. 


In [13]:
ta.add_postprocess_field(pb, "pression_pa", "elem")

In [14]:
print(pb_postprocessing.champs)

format='binaire' mot='dt_post' period='1e6' champs=[Champ_a_post(champ='vitesse', localisation='elem'), Champ_a_post(champ='viscosite_turbulente', localisation='elem'), Champ_a_post(champ='k_eps', localisation='elem'), Champ_a_post(champ='pression_pa', localisation='elem'), Champ_a_post(champ='y_plus', localisation='elem'), Champ_a_post(champ='pression_pa', localisation='elem')]


### Definition champs

Definition champs is a list so we can access to the model fields of an element of the list to see how it is built.

In [15]:
pb_postprocessing.definition_champs[0].model_fields

{'name': FieldInfo(annotation=str, required=False, default='', description='The name of the new created field.'),
 'champ_generique': FieldInfo(annotation=Champ_generique_base, required=False, default_factory=<lambda>, description='not_set')}

We need to define a champ_generique, we can see all the existing class to use the one adapted to what we want to do:

In [ ]:
ta.get_subclass("Champ_generique_base")

We can check informations about the reduction 0d class and ref champ.

In [8]:
 tgp.Reduction_0d.model_fields

NameError: name 'tgp' is not defined

In [ ]:
 tgp.Refchamp.model_fields

With these informations we can create a Reduction_0d with a Refchamp.

In [9]:
refchamp = tgp.Refchamp(nom_source="pb_champ",pb_champ=tgp.Deuxmots(mot_1="pb",mot_2="vitesse"))

NameError: name 'tgp' is not defined

In [ ]:
new_definition_champs = tgp.Definition_champ(name='vit_min', champ_generique=tgp.Reduction_0d(source=refchamp, sources=None, nom_source=None, source_reference=None, sources_reference=None, methode='min'))

In [ ]:
pb_postprocessing.definition_champs.append(new_definition_champs)

In [10]:
ta.write_data(dataset,"Cas_turbulence_modified")

NameError: name 'dataset' is not defined